In [2]:
import requests

OSRM_URL = "http://localhost:5000"

import pandas as pd
df = pd.read_csv('train_cleaned.csv', index_col=0)

In [4]:
def get_osrm_route(start_lat, start_lng, end_lat, end_lng):
    """Fetch route details from OSRM based on start and end coordinates."""
    coordinates = f"{start_lng},{start_lat};{end_lng},{end_lat}"
    route_url = f"{OSRM_URL}/route/v1/driving/{coordinates}?overview=false"  # Change 'overview' if full route is needed

    response = requests.get(route_url)
    
    if response.status_code == 200:
        data = response.json()
        if data["routes"]:  # Ensure routes are returned
            return data["routes"][0]["distance"], data["routes"][0]["duration"]
    return None, None  # Return None if no valid response

# Filter the DataFrame to include only rows where start_city is 'SF'
sf_df = df[df['start_city'] == 'SF']
sf_df


,row_id,start_lng,start_lat,end_lng,end_lat,datetime,duration,missing_coords,start_city,end_city,...,bearing,hour,day_of_week,month,hour_sin,hour_cos,day_of_week_sin,day_of_week_cos,month_sin,month_cos
1,1,-122.422385,37.789408,-122.419890,37.766692,2012-09-14 01:46:00,300.0,False,SF,SF,...,175.016926,1,4,9,0.258819,9.659258e-01,-0.433884,-0.900969,-1.0,-1.836970e-16
3,3,-122.401603,37.788975,-122.410958,37.796447,2012-09-03 23:48:00,360.0,False,SF,SF,...,315.188487,23,0,9,-0.258819,9.659258e-01,0.000000,1.000000,-1.0,-1.836970e-16
6,6,-122.422805,37.772178,-122.396112,37.757865,2012-09-07 23:58:00,720.0,False,SF,SF,...,124.028698,23,4,9,-0.258819,9.659258e-01,-0.433884,-0.900969,-1.0,-1.836970e-16
9,9,-122.405385,37.789718,-122.410163,37.808155,2012-09-08 17:21:00,840.0,False,SF,SF,...,348.381400,17,5,9,-0.965926,-2.588190e-01,-0.974928,-0.222521,-1.0,-1.836970e-16
10,10,-122.403160,37.802698,-122.407443,37.784335,2012-09-04 11:22:00,480.0,False,SF,SF,...,190.487417,11,1,9,0.258819,-9.659258e-01,0.781831,0.623490,-1.0,-1.836970e-16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145979,145979,-122.405147,37.784908,-122.413423,37.804387,2012-09-12 13:03:00,540.0,False,SF,SF,...,341.371071,13,2,9,-0.258819,-9.659258e-01,0.974928,-0.222521,-1.0,-1.836970e-16
145981,145981,-122.409657,37.806140,-122.409353,37.801245,2012-09-11 04:22:00,120.0,False,SF,SF,...,177.178886,4,1,9,0.866025,5.000000e-01,0.781831,0.623490,-1.0,-1.836970e-16
145984,145984,-122.395787,37.776412,-122.404120,37.791180,2012-09-12 07:23:00,300.0,False,SF,SF,...,335.878363,7,2,9,0.965926,-2.588190e-01,0.974928,-0.222521,-1.0,-1.836970e-16
145989,145989,-122.394238,37.780037,-122.397173,37.784287,2012-09-03 18:07:00,120.0,False,SF,SF,...,331.273697,18,0,9,-1.000000,-1.836970e-16,0.000000,1.000000,-1.0,-1.836970e-16


In [5]:
# Apply the function to the filtered DataFrame
sf_df["osrm_distance"], sf_df["osrm_duration"] = zip(*sf_df.apply(lambda row: get_osrm_route(
    row["start_lat"], row["start_lng"], row["end_lat"], row["end_lng"]), axis=1))



/tmp/ipykernel_23094/2583025916.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sf_df["osrm_distance"], sf_df["osrm_duration"] = zip(*sf_df.apply(lambda row: get_osrm_route(
/tmp/ipykernel_23094/2583025916.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sf_df["osrm_distance"], sf_df["osrm_duration"] = zip(*sf_df.apply(lambda row: get_osrm_route(


In [6]:
sf_df[['osrm_duration', 'osrm_distance']]

,osrm_duration,osrm_distance
1,340.0,2718.3
3,237.7,2050.5
6,407.6,3900.5
9,319.1,2816.4
10,308.2,3119.4
...,...,...
145979,393.2,3617.9
145981,117.2,892.4
145984,189.9,2137.6
145989,75.4,804.7


In [9]:
sf_df[['osrm_duration', 'osrm_distance']].isnull().sum()

osrm_duration    0
osrm_distance    0
dtype: int64

In [10]:
sf_df[['osrm_duration', 'osrm_distance']].describe()

,osrm_duration,osrm_distance
count,40001.000000,40001.000000
mean,503.276396,6245.874578
std,414.263132,7606.398363
min,11.600000,103.800000
25%,234.500000,2025.300000
50%,360.100000,3277.300000
75%,584.100000,5851.500000
max,6240.900000,136512.100000


In [7]:
sf_df.to_csv('sf_df_osmr.csv', index=False)